# **Bank of Italy (Asia-Pacific)-API & OpenAI integrated text software-DATA IMPORT**

Author: Elia Landini______Edition: Definitive______AI version: OpenAI 3.5

### **PRELIMINARY SETUPS**

In [1]:
!pip install openai 
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install reportlab
!pip install schedule
!pip install requests-html
!pip install jpstat
!pip install wbgapi
!pip install fredapi
!pip install lxml 

### **1) DATA IMPORT**

##### 1.1) e-Stat: Japanese Goverment Statistics 

In [85]:
import os
import pandas as pd
from requests_html import HTMLSession
import jpstat
import json
import sys
import re

In [3]:
# Function: scrape_stat
# helper functions (jpstat) to get the list of the available macro-variables' ID on e-Stat
# the @ID column does not provide what will be lately called "statsDataId" into the get_data function

def scrape_stat():
    url_base = 'https://www.e-stat.go.jp/en/stat-search/files?page=1'
    def url_info_base(
        x): return f"https://www.e-stat.go.jp/retrieve/api_file_modal?params[filters][toukei_cd]={x}&params[depth]=1&params[lang]=en"

    session = HTMLSession()
    r = session.get(url_base)
    page_info = r.html.find(".stat-paginate-index", first=True)
    if page_info:
        pages = int(page_info.text[2:-3])
    else:
        pages = 1

    stats = []

    for p in range(pages):
        url = url_base[:-1] + str(p+1)
        r = session.get(url)

        tables = r.html.find(".stat-search_result-item1-main")
        for t in tables:
            d = {}
            d["@id"] = t.find(".stat-toukei_code_items > .stat-title", first=True).text
            d["STAT_NAME"] = t.find(".stat-toukei_name_items > .stat-title", first=True).text
            if t.find(".fa.fa-info", first=True):
                url_info = url_info_base(d["@id"])
                r = session.get(url_info)
                info = json.loads(r.html.text)
                d["GOV_ORG"] = info["kikan_kashitsu"]
                d["EXPLANATION"] = info["explanation"]
                d["HP_URL"] = info["exp_url"]

            stats.append(d)

        sys.stdout.write(".")
        sys.stdout.flush()

    df = pd.DataFrame(stats)
    return df
dfstat = scrape_stat()
print(dfstat)

.         @id                                          STAT_NAME
0   00100406                     Indexes of Business Conditions
1   00100409                                  National Accounts
2   00130002                      Statistics about Road Traffic
3   00200502        System of Social and Demographic Statistics
4   00200521                                  Population Census
5   00200522                            Housing and Land Survey
6   00200523              Report on Internal Migration in Japan
7   00200524                               Population Estimates
8   00200531                                Labour Force Survey
9   00200532                           Employment Status Survey
10  00200533          Survey on Time Use and Leisure Activities
11  00200541                   Unincorporated Enterprise Survey
12  00200544               Monthly Survey on Service Industries
13  00200545                       Survey on Service Industries
14  00200551                Establishme

In [4]:
# Function: scrape_list
# helper functions (jpstat) to search data by either the code of a statistic or some keywords

def scrape_list(statsCode, year=None):
    url_base = f"https://www.e-stat.go.jp/en/stat-search/files?layout=dataset&toukei={statsCode}"
    if year:
        url_base += f"&year={year}0"

    session = HTMLSession()
    r = session.get(url_base)
    page_info = r.html.find(".stat-paginate-index", first=True)
    if page_info:
        pages = int(page_info.text[2:-4])
    else:
        pages = 1
    url_base = url_base + "&page="

    stats = []

    for p in range(pages):
        url = url_base + str(p+1)
        r = session.get(url)

        tables = r.html.find(".stat-resource_list-main")
        for t in tables:
            d = {}
            info = t.find(".stat-resource_list-detail-item")
            d["STAT_NAME"] = info[0].text
            d["STAT_CAT"] = info[1].text
            d["SURVEY_DATE"] = info[2].text.replace("調査年月", "").replace("\xa0", "")
            d["OPEN_DATE"] = info[3].text.replace("公開（更新）日", "").replace("\xa0", "")
            file_links = info[4].find("a")
            for fl in file_links:
                if "data-file_type" not in fl.attrs.keys():
                    d["API"] = True
                    d["API_@id"] = list(fl.links)[0].replace("../dbview?sid=", "")
                else:
                    file_type = fl.attrs["data-file_type"]
                    d[file_type] = True
            data = t.find(".stat-link_text", first=True)
            d["@id"] = data.attrs["data-value"]
            d["STATISTICS_NAME"] = data.find(".stat-resource_list-detail-item-text", first=True).text.replace("\u3000", "-")

            stats.append(d)

        sys.stdout.write(".")
        sys.stdout.flush()

    df = pd.DataFrame(stats)
    return df

In [5]:
# jpstat setting
# DISCLAIMER: the so appID is strictly personal and you should refrain from sharing it with strangers 

# API key/appID e-Stat setting
# you can generate up to 3 appId on https://www.e-stat.go.jp/en by registrating a valid account (specific section: "API function (application ID issuance)" in "My page")
# appID-example: 891d7fa0a7dc013ae13e2c2e5264a7f88bfc0d4c

jpstat.options["estat.api_key"] = "891d7fa0a7dc013ae13e2c2e5264a7f88bfc0d4c"

# to see a list of valid configuration options
## jpstat.config.describe_options()

In [10]:
# 1) Data retrieval by API
# statDataId-example: 0003350739
# statURL-example: https://www.e-stat.go.jp/en/dbview?sid=0003350739

# datajp, note = jpstat.estat.get_data(statsDataId="0003350739")
# print(datajp)

# in case of error 
## datajp = jpstat.estat.get_data(statsDataId="0003350739", return_note=False)


# 2) Data retrieval by file
# Alternative to API key's method is the classic file download approach from e-Stat, which, however, may provide oudated data sources
# statURL-example: https://www.e-stat.go.jp/en/dbview?sid=0003350739

##jpstat.estatFile.get_file(statsDataId="0003350739", file_type="EXCEL")

1.1.1) Consumer Price Index 2020-Base Consumer Price Index

In [ ]:
# Retrieving data for Consumer Price Index 2020-Base Consumer Price Index
# https://www.e-stat.go.jp/en/dbview?sid=0003427113 
# frequency: monthly
# dataframe tag: I

dfI, note = jpstat.estat.get_data(statsDataId="0003427113")
print(dfI.head())

In [150]:
# Data manipulation 

# get the columns names 
dfI_column_names = dfI.columns
print([dfI_column_names])

# We want to drop useless columns and rows to come with a clear, and yet comprehensive, final dataframe
# keep only national level observations
dfI_alljapan = dfI[dfI["Area(2020-base)"].str.contains("All Japan")]

# keep only specific disaggregated variables 
dfI1 = dfI_alljapan[dfI_alljapan["Items(2020-base)"].str.contains("All items|Food|Fuel, light & water charges|Electricity|Gas|Water & Sewerage charges|Transportation & Communication|All items, less fresh food and energy, seasonally adjusted|Goods, seasonally adjusted|Services, seasonally adjusted")]

# drop undeleted rows 
phrases_to_drop = [
    "Gas, manufactured & piped",
    "Gas cooking tables",
    "Food wrap",
    "Gastrointestinal medicines",
    "Gasoline",
    "All items, less fresh food",
    "Food, less fresh food",
    "All items, less imputed rent",
    "All items, less imputed rent & fresh food",
    "All items, less fresh food and energy",
    "All items, less food (less alcoholic beverages) and energy",
    "Food products",
    "All items, less fresh food, seasonally adjusted",
]

dfI1 = dfI1[~dfI1["Items(2020-base)"].isin(phrases_to_drop)]
boolean_mask2 = dfI1["Tabulated variable"].str.contains("Change over the year", case=False, na=False)
dfI2 = dfI1.loc[boolean_mask2]

# adapt the time format to MM/YYYY
# not all the values are associated to a %b. %Y format, but some (May) are formatted on a %b %Y structure
def correct_dateformat(date):
    pattern = r"^[a-zA-Z]{3} \d{4}$"
    
    if re.match(pattern, date):
        corrected_date = date[:3] + ". " + date[4:]
        return corrected_date
    else:
        return date 
    
dfI2["Time"] = dfI2["Time"].apply(correct_dateformat)
dfI2["Time"] = pd.to_datetime(dfI2["Time"], format="%b. %Y", errors="coerce")
dfI2["Time"] = dfI2["Time"].dt.strftime("%m/%Y")
dfI3 = dfI2

# reset the index
dfI3.reset_index(drop=True, inplace=True)

print(dfI3)

[Index(['@unit', 'Tabulated variable', 'Items(2020-base)', 'Area(2020-base)',
       'Time', 'Value'],
      dtype='object')]
     @unit    Tabulated variable  \
0        %  Change over the year   
1        %  Change over the year   
2        %  Change over the year   
3        %  Change over the year   
4        %  Change over the year   
...    ...                   ...   
5115     %  Change over the year   
5116     %  Change over the year   
5117     %  Change over the year   
5118     %  Change over the year   
5119     %  Change over the year   

                                       Items(2020-base) Area(2020-base)  \
0                                             All items       All Japan   
1                                             All items       All Japan   
2                                             All items       All Japan   
3                                             All items       All Japan   
4                                             All items       All 

C:\Users\eland\AppData\Local\Temp\ipykernel_19304\747478137.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfI2["Time"] = dfI2["Time"].apply(correct_dateformat)
C:\Users\eland\AppData\Local\Temp\ipykernel_19304\747478137.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfI2["Time"] = pd.to_datetime(dfI2["Time"], format="%b. %Y", errors="coerce")
C:\Users\eland\AppData\Local\Temp\ipykernel_19304\747478137.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [152]:
# Tabulation
dfI4 = {
    "Time": dfI3["Time"], 
    "Inflation Rate (% change over the year)": dfI3["Value"],  
    "Category" : dfI3["Items(2020-base)"]   
}

df_IR = pd.DataFrame(dfI4)
df_IR = df_IR.set_index("Time")

df_IR.head(20)

,Category,Inflation Rate (% change over the year)
Time,,
04/2024,All items,2.5
03/2024,All items,2.7
02/2024,All items,2.8
01/2024,All items,2.2
12/2023,All items,2.6
11/2023,All items,2.8
10/2023,All items,3.3
09/2023,All items,3.0
08/2023,All items,3.2


##### 1.2) World Bank Statistics

In [6]:
import os
import wbgapi as wb
import pandas as pd

In [7]:
# Data retrieval by API
# importing data from the World Bank through API keys is quite straightforward compared to the Japanese counterpart, since official Python-plug ins for this purpose have been published
# usually it is better to apply the ".transpose()" function to the World Bank data's original formatting
# statcode-example: NY.GDP.MKTP.CD
# countrycode-example: JPN
# statURL-example: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=JP 

## datawb = wb.data.DataFrame("NY.GDP.MKTP.CD", "JPN").transpose()
## print(datawb)

# to search for a spefic stat's code 
# statname-example: population
## datacode = wb.search("population")
## print(datacode)

1.2.1) GDP (current US$) - Japan

In [15]:
# Retrieving data for GDP (current US$) - Japan
# https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=JP 
# frequency: yearly
# dataframe tag: Y

dfY = wb.data.DataFrame("NY.GDP.MKTP.CD", "JPN").transpose()
print(dfY.head())

economy           JPN
YR1960   4.430734e+10
YR1961   5.350862e+10
YR1962   6.072302e+10
YR1963   6.949813e+10
YR1964   8.174901e+10


In [95]:
# Data manipulation 

# get the columns names 
dfY_column_names = dfY.columns
print([dfY_column_names])

# rename columns to better fit our vocabulary
dfY1 = dfY.rename(columns={"JPN": "GDP"})

# reset the index, making "economy" a regular column and assign to that new column the name "Year" 
dfY1.reset_index(inplace=True) 
dfY1.rename(columns={"index": "Time"}, inplace=True) 

# modify the time format, by extracting the year part from the "Year" column
dfY1["Time"] = dfY1["Time"].str.slice(start=2) 

print(dfY1)

[Index(['JPN'], dtype='object', name='economy')]
economy  Time           GDP
0        1960  4.430734e+10
1        1961  5.350862e+10
2        1962  6.072302e+10
3        1963  6.949813e+10
4        1964  8.174901e+10
..        ...           ...
59       2019  5.117994e+12
60       2020  5.055587e+12
61       2021  5.034621e+12
62       2022  4.256411e+12
63       2023           NaN

[64 rows x 2 columns]


In [108]:
# Tabulation
dfY2 = {
    "Time": dfY1["Time"],  
    "GDP (current US$)": dfY1["GDP"],    
}

df_GDP = pd.DataFrame(dfY2)
df_GDP = df_GDP.sort_values(by="Time", ascending=False)
df_GDP = df_GDP.set_index("Time")

df_GDP.head(20)

,GDP (current US$)
Time,
2023,NaN
2022,4.256411e+12
2021,5.034621e+12
2020,5.055587e+12
2019,5.117994e+12
2018,5.040881e+12
2017,4.930837e+12
2016,5.003678e+12
2015,4.444931e+12


##### 1.3) FRED: Federal Reserve Economic Data

In [13]:
import os
import fredapi as fa
import pandas as pd
from fredapi import Fred

In [14]:
# freadpi setting
# DISCLAIMER: the so API key is strictly personal and you should refrain from sharing it with strangers 

# FRED API key  setting
# you can generate an API key on https://fred.stlouisfed.org/docs/api/api_key.html by registrating a valid account
# FRED API key-example: a8f0cdf1d0b64d1188b9c64ed64c77b6

fred = Fred(api_key="a8f0cdf1d0b64d1188b9c64ed64c77b6")

In [9]:
# Data retrieval by API
# quite similar approach to what we have seen for wbgapi
# statcode-example: EXJPUS
# statURL-example: https://fred.stlouisfed.org/series/EXJPUS 

## datafred = fred.get_series("EXJPUS")
## print(datafred.head())

# with FRED data is often necessary to convert the serie in a proper dataframe
## datafred = datafred.to_frame()

# to get latest data  
## datafredlatest = fred.get_series_latest_release("EXJPUS")
## print(datafredlatest.head())

# to get latest data known on a given date
# releasedate-example: 7/10/2015
## datafredlatestknown = fred.get_series_as_of_date("EXJPUS", "7/10/2015")
## print(datafredlatestknown.head())

# fredapi also includes the option to retrieve variables' data by typing their names
# statname-example: GDP
## datafredname = fred.get_series("GDP")
## print(datafredname.head())

1.3.1) Japanese Yen to U.S. Dollar Spot Exchange Rate 

In [17]:
# Retrieving data for Japanese Yen to U.S. Dollar Spot Exchange Rate 
# https://fred.stlouisfed.org/series/EXJPUS 
# frequency: monthly
# dataframe tag: yen

dfyen = fred.get_series("EXJPUS")
dfyen = dfyen.to_frame()
print(dfyen.head())

                   0
1971-01-01  358.0200
1971-02-01  357.5450
1971-03-01  357.5187
1971-04-01  357.5032
1971-05-01  357.4130


In [122]:
# Data manipulation 

# get the columns names 
dfyen_column_names = dfyen.columns
print([dfyen_column_names])

# rename columns to better fit our vocabulary
dfyen1 = dfyen.rename(columns={dfyen.columns[0]: "JPY/USD"})

# reset the index, making the time index a regular column and assign to that new column the name "Year"
dfyen1.reset_index(inplace=True) 
dfyen1.rename(columns={"index": "Time"}, inplace=True) 

# adapt the time format to MM/YYY
dfyen1["Time"] = pd.to_datetime(dfyen1["Time"], format="%m/%Y", errors="coerce")
dfyen2 = dfyen1

print(dfyen2)

[RangeIndex(start=0, stop=1, step=1)]
          Time   JPY/USD
0   1971-01-01  358.0200
1   1971-02-01  357.5450
2   1971-03-01  357.5187
3   1971-04-01  357.5032
4   1971-05-01  357.4130
..         ...       ...
636 2024-01-01  146.2943
637 2024-02-01  149.6150
638 2024-03-01  149.8186
639 2024-04-01  153.8900
640 2024-05-01  155.8691

[641 rows x 2 columns]


In [123]:
# Tabulation
dfyen3 = {
    "Time": dfyen2["Time"],  
    "JPY/USD Spot Exchange Rate": dfyen2["JPY/USD"],    
}

df_exy = pd.DataFrame(dfyen3)
df_exy = df_exy.sort_values(by="Time", ascending=False)
df_exy["Time"] = df_exy["Time"].dt.strftime("%m/%Y")
df_exy = df_exy.set_index("Time")

df_exy.head(20)

,JPY/USD Spot Exchange Rate
Time,
05/2024,155.8691
04/2024,153.8900
03/2024,149.8186
02/2024,149.6150
01/2024,146.2943
12/2023,143.9815
11/2023,149.6790
10/2023,149.5933
09/2023,147.8450
